In [10]:
#just need to extract the time for the LED pulse and produce a CSV with this in.
#LED square pulse should be going up to 2.9V but triggered when 960mV - find time on oscilloscope when this is true and the index of this

In [11]:
import sys
import time
import matplotlib.pyplot as plt 
import numpy as np 
import tekwfm
import os
from os.path import exists
from scipy import signal
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.special import gamma
import matplotlib.cm as cm
from collections import Counter

import gc
import re
import tempfile
from collections import defaultdict, namedtuple
from pathlib import Path
import pandas as pd

In [12]:
doVerbose = False

#import the wfm file
filename_base = "030226_spe_1750_500_1_on_ch2"


#saving the processed file

folder_name = "PMT7_SPE_CLEAN"
file_name = "030226_spe_1750_500_1_on_ch2_processed.csv" 
file_path = os.path.join(os.getcwd(), folder_name, file_name)


startEvent = 0
nEvents = 500


volts, tstart, tscale, tfrac, tdatefrac, tdate = tekwfm.read_wfm(filename_base+".wfm")
if doVerbose:
    print('>>>>', volts, tstart, tscale, tfrac, tdatefrac, tdate)

samples = volts.shape

print(len(volts))
print(tscale)
print('samples', samples[0])
tstop = samples[0]*tscale+tstart
sampleTimes = [tstart+x*tscale for x in range(samples[0])]

#already positive, don't need to flip like with CH1.
tempVolts = [volts[:, event] for event in range(startEvent, startEvent + nEvents)] # all events, array of array of voltages



12500
8e-11
samples 12500


In [13]:
def first_crossing_time(tempVolts, sampleTimes, threshold=0.960):

    for i in range(1, len(tempVolts)):
        if tempVolts[i-1] < threshold <= tempVolts[i]:
            #linear interpolation
            y1, y2 = tempVolts[i-1], tempVolts[i]
            x1, x2 = sampleTimes[i-1], sampleTimes[i]
    
            if (y2 - y1)==0:
                return None, None, None
            frac = (threshold - y1) / (y2 - y1)
            if not (0<=frac<=1):
                return None, None, None
        
            first_cross = x1 + frac * (x2 - x1)

            return first_cross
    
    #if no crossing found
    return None

def cross_index(first_cross, sampleTimes):
    if first_cross is None:
        return len(sampleTimes) - 1
    else:
        time_difference = np.abs(np.array(sampleTimes) - first_cross)
        cross_index = np.argmin(time_difference)
        return cross_index



In [14]:
#for all events
cross_times = []
cross_indices = []

for tempVolt in tempVolts:
    t_cross = first_crossing_time(tempVolt, sampleTimes)
    i_cross = cross_index(t_cross, sampleTimes)
    cross_times.append(t_cross)
    cross_indices.append(i_cross)

print(cross_times)
print(cross_indices)

[None, None, np.float64(-1.3275333333333334e-07), np.float64(-3.5875999999999996e-07), np.float64(-6.203294117647059e-08), np.float64(-3.321672727272727e-07), None, np.float64(-3.8388888888888884e-07), np.float64(-1.5881777777777778e-07), np.float64(-1.9810879999999998e-07), np.float64(-2.190507692307692e-07), np.float64(-1.517651063829787e-07), np.float64(-2.861395348837196e-09), None, np.float64(-1.9595733333333333e-07), np.float64(-1.7055999999999994e-08), np.float64(-6.360000000000007e-09), np.float64(-5.1786666666666657e-08), np.float64(-2.9277037037037036e-07), np.float64(-3.5735999999999997e-07), np.float64(-3.077670588235294e-07), None, np.float64(-2.1315478260869565e-07), np.float64(-3.6176941176470587e-07), np.float64(-1.1210133333333332e-07), None, np.float64(-1.0240307692307694e-07), np.float64(-6.447457627118639e-09), np.float64(-1.697184e-07), None, np.float64(-3.310571428571428e-07), np.float64(-7.880190476190475e-08), None, np.float64(-1.6851294117647057e-07), np.float6

In [15]:
#produce the csv file

df = pd.DataFrame({
            'crossing_trigger': cross_times,
            'crossing_index': cross_indices,
            })

df.to_csv(file_path, sep=',', encoding='utf-8-sig', index=True, header=True)

print('CSV done!')

CSV done!
